In [51]:
import copy
import json

filename = "./extracted_skills/original_predictions/knowledge.tech.test.3044792.out"

In [31]:
def load_bio_to_json(file_path):
    default_result = {
        "tokens": [],
        "tags_skill": [],
        "tags_knowledge": []
    }
    
    results = []
    
    current_result = copy.deepcopy(default_result);
    
    with open(file_path, 'r') as file:
        for line in file:
            cleaned_line = line.strip()
            
            if (cleaned_line == ''):
                if (len(current_result["tokens"]) != 0):
                    results.append(current_result);
                    current_result = copy.deepcopy(default_result);
            else:
                splitten_line = cleaned_line.split("\t");
                current_result["tokens"].append(splitten_line[0])
                current_result["tags_skill"].append(splitten_line[1][0])
                current_result["tags_knowledge"].append(splitten_line[2][0])
                
    return results;

In [33]:
results = load_bio_to_json(filename)

results[0:2]

[{'tokens': ['Full',
   'Stack',
   'Software',
   'Engineer',
   '-',
   'Java',
   '/',
   'JavaScript'],
  'tags_skill': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
  'tags_knowledge': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B']},
 {'tokens': ['<ORGANIZATION>',
   '<ORGANIZATION>',
   '<ORGANIZATION>',
   '<ORGANIZATION>',
   '.'],
  'tags_skill': ['O', 'O', 'O', 'O', 'O'],
  'tags_knowledge': ['O', 'O', 'O', 'O', 'O']}]

In [42]:
original_tech_skills = "./extracted_skills/original_predictions/skills.tech.test.3044792.out"
original_tech_knowledge = "./extracted_skills/original_predictions/knowledge.tech.test.3044792.out"

def load_and_merge(skills_path, knowledge_path):
    skills_preds = load_bio_to_json(skills_path)
    knowledge_preds = load_bio_to_json(knowledge_path)
    
    merged_preds = []
    
    for index, item in enumerate(skills_preds):
        current_skills_pred = skills_preds[index]
        current_knowledge_pred = knowledge_preds[index]
        
        merged_preds.append({
            'tokens': current_skills_pred['tokens'],
            'tags_skill': current_skills_pred["tags_skill"],
            'tags_knowledge': current_knowledge_pred["tags_knowledge"]
        })
        
    return merged_preds

original_tech_merged = load_and_merge(original_tech_skills, original_tech_knowledge)

original_tech_merged[:3]

[{'tokens': ['Full',
   'Stack',
   'Software',
   'Engineer',
   '-',
   'Java',
   '/',
   'JavaScript'],
  'tags_skill': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
  'tags_knowledge': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B']},
 {'tokens': ['<ORGANIZATION>',
   '<ORGANIZATION>',
   '<ORGANIZATION>',
   '<ORGANIZATION>',
   '.'],
  'tags_skill': ['O', 'O', 'O', 'O', 'O'],
  'tags_knowledge': ['O', 'O', 'O', 'O', 'O']},
 {'tokens': ['<ADDRESS>', '<ADDRESS>', '<LOCATION>', '-', '<LOCATION>'],
  'tags_skill': ['O', 'O', 'O', 'O', 'O'],
  'tags_knowledge': ['O', 'O', 'O', 'O', 'O']}]

In [44]:
paths = {
    "original_tech_skills": "./extracted_skills/original_predictions/skills.tech.test.3044792.out",
    "original_tech_knowledge": "./extracted_skills/original_predictions/knowledge.tech.test.3044792.out",
    "original_house_skills": "./extracted_skills/original_predictions/skills.house.test.3044792.out",
    "original_house_knowledge": "./extracted_skills/original_predictions/knowledge.house.test.3044792.out",
    "enriched_tech_skills": "./extracted_skills/enriched_predictions/skills_enriched.tech.test.3044792.out",
    "enriched_tech_knowledge": "./extracted_skills/enriched_predictions/knowledge_enriched.tech.test.3044792.out",
    "enriched_house_skills": "./extracted_skills/enriched_predictions/skills_enriched.house.test.3044792.out",
    "enriched_house_knowledge": "./extracted_skills/enriched_predictions/knowledge_enriched.house.test.3044792.out",
}

predictions_data = {}

predictions_data["original_tech"] = load_and_merge(paths["original_tech_skills"], paths["original_tech_knowledge"])
predictions_data["original_house"] = load_and_merge(paths["original_house_skills"], paths["original_house_knowledge"])
predictions_data["enriched_tech"] = load_and_merge(paths["enriched_tech_skills"], paths["enriched_tech_knowledge"])
predictions_data["enriched_house"] = load_and_merge(paths["enriched_house_skills"], paths["enriched_house_knowledge"])

originial

In [70]:
gold_data = {}

with open("./data/SkillSpan/json/test.json", 'r') as file:
    json_data = json.load(file)
    gold_data["tech"] = [item for item in json_data if item["source"] == "tech"]
    gold_data["house"] = [item for item in json_data if item["source"] == "house"]

In [71]:
print(len(gold_data["house"]))
print(len(gold_data["tech"]))
print(len(predictions_data["enriched_house"]))
print(len(predictions_data["enriched_tech"]))

1221
2349
1221
2349


In [93]:
def is_equal(list1, list2):
    return Counter(list1) == Counter(list2)

In [96]:
from collections import Counter

house_mismatches = []
tech_mismatches = []

for index, gold_item in enumerate(gold_data["house"]):
    original_pred_item = predictions_data["original_house"][index]
    enriched_pred_item = predictions_data["enriched_house"][index]
    
    if(
        (not is_equal(original_pred_item["tags_skill"], enriched_pred_item["tags_skill"])) or
        (not is_equal(original_pred_item["tags_knowledge"], enriched_pred_item["tags_knowledge"]))
    ):
        house_mismatches.append({
            "gold": gold_item,
            "original": original_pred_item,
            "enriched": enriched_pred_item
        })
        
for index, gold_item in enumerate(gold_data["tech"]):
    original_pred_item = predictions_data["original_tech"][index]
    enriched_pred_item = predictions_data["enriched_tech"][index]
    
    if(
        (not is_equal(original_pred_item["tags_skill"], enriched_pred_item["tags_skill"])) or
        (not is_equal(original_pred_item["tags_knowledge"], enriched_pred_item["tags_knowledge"]))
    ):
        tech_mismatches.append({
            "gold": gold_item,
            "original": original_pred_item,
            "enriched": enriched_pred_item
        })
        
print(len(house_mismatches))
print(len(tech_mismatches))

252
303


In [102]:
with open("./extracted_skills/mismatches/house.json", 'w') as f:
    json.dump(house_mismatches, f)

In [103]:
with open("./extracted_skills/mismatches/tech.json", 'w') as f:
    json.dump(tech_mismatches, f)

In [4]:
from fast_bleu import BLEU, SelfBLEU


ref1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
          'ensures', 'that', 'the', 'military', 'will', 'forever',
          'heed', 'Party', 'commands']

ref2 = ['It', 'is', 'the', 'guiding', 'principle', 'which',
          'guarantees', 'the', 'military', 'forces', 'always',
          'being', 'under', 'the', 'command', 'of', 'the', 'Party']

ref3 = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
         'army', 'always', 'to', 'heed', 'the', 'directions',
          'of', 'the', 'party']



list_of_references = [ref1, ref2, ref3]

weights = {'bigram': (1/2., 1/2.), 'trigram': (1/3., 1/3., 1/3.)}

self_bleu = SelfBLEU(list_of_references, weights)

self_bleu.get_score()

{'bigram': [0.25819888974716115, 0.3615507630310936, 0.37080992435478316],
 'trigram': [0.07808966062765045, 0.20140620205719253, 0.21415334758254043]}